# Dashboard Project
This is a project that starts as a simple script and ends as a professional-grade trading and data analysis platform.

## Phase 1 (Jan - Mar)
**The Goal**: Just get data.\
**What I will build**: A script that asks for a stock ticker (like "AAPL") and fetches its current price using a free API (like Yahoo Finance or Alpha Vantage).\
**What I will learn**: Variables, user input, if/else logic, and how to use APIs (how programs talk to each other).

In [ ]:
import yfinance as yf
import csv
import pandas as pd
import matplotlib.pyplot as plt
import os

def ensure_output_dir():
    if not os.path.exists('output'):
        os.makedirs('output')

def get_price(data):
    try:
        price = data['Close'].iloc[-1]
        return price
    except (KeyError, Exception) as e:
        return None

# WHen the user exits its saves history list to a CSV file (e.g., output/stock_log.csv).
def stock_logger_csv(history):
    ensure_output_dir()
    with open('output/stock_log.csv', 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['Ticker', 'Price'])
        for ticker in history:
            csv_writer.writerow([ticker[0].ticker, ticker[1]])

def generate_chart():
    try:
        df = pd.read_csv('output/stock_log.csv')
        if df.empty:
            print('No data to plot.')
            return
        top_5 = df.nlargest(5, 'Price')
        tickers = top_5['Ticker']
        prices = top_5['Price']
        # generate chart
        plt.bar(tickers, prices, width=0.5, color='green')
        plt.xlabel('Stock Ticker')
        plt.ylabel('Price in USD ($)')
        plt.title('Stock Price Watchlist')
        # display the graph (save instead of show to avoid GUI backend blocking)
        plt.tight_layout()
        ensure_output_dir()
        plt.savefig('output/chart.png')
        plt.close()
        print("Saved bar chart to 'output/chart.png'.")
    except FileNotFoundError:
        print('No data file found.')


def plot_history(data, user_input):
    print(data.head())
    sma_5 = data['Close'].rolling(window=5).mean()
    # print days where the close price is greater than the 5-day SMA
    buy_signals = data['Close'] > sma_5
    plot_signals = data[buy_signals]
    print("Buy signals:")
    print(data[buy_signals])
    # Graphing
    plt.figure(figsize=(10, 10))
    plt.plot(data['Close'], label='Price', color='blue')
    plt.plot(sma_5, label='5-day SMA', color='red')
    plt.scatter(plot_signals.index, plot_signals['Close'], color='green', label='Buy Signals', marker='^', zorder=3)
    plt.title(f'{user_input} Price History')
    plt.xlabel('Date')
    plt.xticks(rotation=45)
    plt.ylabel('Price ($)')
    plt.legend()
    # Save instead of blocking GUI show to avoid macOS backend hang
    plt.tight_layout()
    ensure_output_dir()
    out_path = os.path.join('output', f"{user_input}_history.png")
    plt.savefig(out_path)
    plt.close()
    print(f"Saved history chart to '{out_path}'.")

def main():
    # enter stock ticker
    print("Welcome to the Alpha Dashboard!")
    print("===============================")
    history = []
    while True:
        user_input = input("Please enter a stock ticker (eg. AAPL, MSFT): ")
        user_input = user_input.upper().strip()
        if user_input == "EXIT" or user_input == "Q":
            print("Thank you for using this stock checker.")
            print("History")
            print("---------")
            for ticker in history:
                print(f"{ticker[0].ticker}: {ticker[1]:.2f}")
            stock_logger_csv(history)
            generate_chart()
            break

        ticker = yf.Ticker(user_input)
        try:
            # Try to access ticker info to validate
            data = ticker.history(period='1mo')
            price = get_price(data)
            if price is None:
                print(f"Error: Price data unavailable for '{user_input}'")
            else:
                print(f"{ticker.ticker}: ${price:.2f}")
                history.append([ticker, price])
                show_plot = input("Plot history? (y/n): ").lower()
                if show_plot == 'y':
                    plot_history(data, user_input)
                    print("Graph generated.")
        except (KeyError, Exception) as e:
            print(f"Error: '{user_input}' is not a valid stock ticker or data is unavailable.\nPlease check the ticker symbol and try again.")